In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# define your transform using transforms.Compose -- this is the way you normalize your data into tensors before putting it into the neural net
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# define batch size as 4
batch_size = 4

# define your trainset using torchvision.datasets.CIFAR10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
# define your test set the same way
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
# define your test loader using torch.utils.data.DataLoader
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
# Define Convolutional Neural Network

import torch.nn as nn
import torch.nn.functional as F

# Define your convolutional neural net class
class Net(nn.Module):
    # initialization function
    def __init__(self):
        super().__init__()
        # initialize conv layer
        self.conv1 = nn.Conv2d(3, 6, 5)
        # initialize maxpool layer
        self.pool = nn.MaxPool2d(2, 2)
        # initialize conv layer
        self.conv2 = nn.Conv2d(6, 16, 5)
        # initialize 3 fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    # forward function
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [5]:
import torch.optim as optim
# define criterion using the cross entropy loss function nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()
# define optimizer optim.SGD
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
# make a for loop for training (for epoch in range(2))
for epoch in range(2):  

    running_loss = 0.0
    # make a for loop to iterate through the trainloader 
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.189
[1,  4000] loss: 1.853
[1,  6000] loss: 1.668
[1,  8000] loss: 1.601
[1, 10000] loss: 1.521
[1, 12000] loss: 1.480
[2,  2000] loss: 1.407
[2,  4000] loss: 1.360
[2,  6000] loss: 1.339
[2,  8000] loss: 1.332
[2, 10000] loss: 1.292
[2, 12000] loss: 1.301
Finished Training


: 

In [ ]:
# save model weights to a path using torch.save
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))